### Este notebook tem como função criar uma pasta chamada "Faces Dataset" onde os rostos recortados dos vídeos serão guardados

Inicialmente, carregamos as dependências necessárias

In [1]:
import cv2

import pandas as pd

from facenet_pytorch import MTCNN

from PIL import Image
import glob, os

import torch
import torchvision
import numpy as np

import random

import matplotlib.pyplot as plt

import time

# Metodologia para criar o Image Dataset:
- Inicialmente, achar bons valores de threshold para a detecção dos rostos, de forma que se garanta uma confiabilidade alta para todos os vídeos. (garantir que estou encontrando o rosto e estou recortando nos limiares aceitáveis). Podem haver imagens salvas que não são rostos, mas se o threshold estiver bem ajustado, não será suficiente para impactar no desempenho da nossa rede.
- Teremos valores móveis para decidir de quantos em quantos frames a detecção será realizada e o rosto recortado salvo.

Em seguida, seguir o seguinte loop de rotina:

Para cada pasta do dataset:

Para cada vídeo dentro da pasta:
- Abrir o vídeo frame a frame.
- Aplicar a detecção do rosto (com tamanho padrão definido) => Tamanho de entrada dos modelos pré treinados = 224x224 px
- Recortar a região devolvida do rosto no canal RGB natural.
- Salvar em uma pasta correspondente sendo FAKE ou REAL
- Repetir

# 1° passo: Definir as funções e os objetos principais de acesso aos vídeos nas pastas.

Define-se o device onde será rodado a detecção de rostos.

In [2]:
# Definimos um device onde os tensores estarão sendo processados
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [3]:
# Cria uma lista de todas as pastas disponíveis para treinamento
folders = next(os.walk('./Kaggle Dataset/'))[1]

Cria-se uma classe para controlar vídeos. Essa classe recebe um folder específico onde contém vídeos, e coleta todos os vídeos ali dentro presentes

In [4]:
# Cuida de lidar com o acesso aos vídeos e devolver os paths / labels corretamente
class Videos():
    def __init__(self, folder_path):
        # Guarda o folder_path
        self.folder_path = folder_path
        
        # Guarda a lista de todos os arquivos de videos dentro do folder_path
        self.video_files = glob.glob(folder_path + '/*.mp4')
        
        # Lê o arquivo JSON que contém as informações dos deepfakes naquela pasta
        self.metadata = pd.read_json(folder_path + '/metadata.json').transpose() # Essa transposiçao eh feita pois as colunas e as linhas estao trocadas
        
    def getRandomVideo(self):
        video_path = random.choice(self.video_files)
        video_name = os.path.basename(video_path)
        label = self.metadata.loc[video_name].label
        
        return video_path, video_name, label
        
    def getRealVideo(self, video_name):
        real_video_name = self.metadata.loc[video_name].original
        # Verifica se é NaN, pois caso seja o nome original é o próprio video real
        if pd.isna(real_video_name):
            real_video_name = video_name
        real_video_path = folder_path + '/' + real_video_name
        return real_video_path, real_video_name, 'REAL'
    
    def getAllVideosPath(self):
        for video_name, columns in self.metadata.iterrows():
            yield self.folder_path + '/' + video_name, video_name, columns[0] # Label

In [5]:
# Esta função permite realiza algo similar à showVideo(), porém ela abre um segundo display de vídeo mostrando a região da face recortada
def showVideo(video_path, label, padding=0, size=-1, channel=None):
    cap = cv2.VideoCapture(video_path)
    
    # Configura a cor a ser colocada na LABEL
    if label == 'REAL':
        color = (0, 255, 0) # Verde
    else:
        color = (0, 0, 255) # Vermelho  
    
    face = None
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            boxes, _ = mtcnn.detect(Image.fromarray(frame))
            if boxes is not None:
                for box in boxes: 
                    face = frame[int(box[1] - padding):int(box[3] + padding), int(box[0] - padding):int(box[2] + padding)].copy()
                    cv2.putText(img=frame, text=label, org=(box[0], box[1]), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=color, thickness=2)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color=[0, 255, 0], thickness=5)
                    if channel == 'luma':
                        face = cv2.cvtColor(face, cv2.COLOR_BGR2YCrCb)
                        face = face[:,:,0] # Pega apenas o canal Y
                    
            if face is not None:
                if size > 0:
                    face = cv2.resize(face, (size, size))
                    
                cv2.imshow('face', face)
                
            frame = cv2.resize(frame, (1280, 720))
            cv2.imshow('frame', frame)
            
            # Apertar a tecla 'q' para sair do vídeo.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        else:
            cv2.destroyAllWindows()
            break

    cap.release()
    cv2.destroyAllWindows()

In [6]:
# Essa é de fato a função que entra no loop e é responsável por recortar o rosto e o salvar 
# na pasta correspondente, para todos os frames do vídeo. Ela é chamada 1 vez por vídeo.
def saveCropFaces(video_path, video_name, label, batch_size=20, padding=0, size=224, check_every_frame=5, channel=None, folder='Dataset provisório', size_folder='no-resize-color'):
    
    # Instancia um VideoCapture do arquivo presente em video_path (no caso, o vídeo)
    cap = cv2.VideoCapture(video_path)
    # Pega, em inteiros, a quantidade de frames do vídeo
    v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Cria um path para salvar os rostos recortados do vídeo. O path nesse caso está sendo './Faces Dataset/224px/FAKE ou REAL'
    class_path = './' + folder + '/' + size_folder + '/' + label
    
    # Inicializa frames como uma lista vazia
    frames = []
 
    # Entra num loop que percorre o vídeo até ele acabar
    for _ in range(1, v_len + 1):
        # Realiza um grab() no próximo frame, mas não o decodifica. Isso ajuda a agilizar o processo se não for necessário
        # recuperar todos os frames a todo o momento.
        success = cap.grab()
        # Só recorta o rosto se o frame atual for mod check_every_frame, ou seja, ele só decodifica de check_every_frame em check_every_frame frames.
        if _ == 1 or _ % check_every_frame == 0:
            success, frame = cap.retrieve()
        else:
            continue
        if not success:
            continue
        # Realiza um append do frame atual na lista frames
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frames.append(frame)
        
        if len(frames) >= batch_size or (_ == v_len and len(frames) > 0):
            # Utiliza o MTCNN para detectar todas as bounding boxes de todos os rostos
            boxes, probs = mtcnn.detect(frames)
            # Verifica se não foi obtida nenhuma bounding box em todo o batch
            if not all(x is None for x in boxes):
                # Acessa cada um dos frames no batch
                for i, boxes_f in enumerate(boxes):
                    # Verifica houve None para o frame atual
                    if boxes_f is not None:
                        # Acessa cada uma das bounding boxes dentro de um único frame (pode haver vários rostos)
                        face_count = 0
                        for bbox in boxes_f:
                            #face = np.array(frames[i])[
                            #    int(max(bbox[1] - padding, 0)):int(max(bbox[3] + padding, 0)), 
                            #    int(max(bbox[0] - padding, 0)):int(max(bbox[2] + padding, 0))
                            #]
                            face = frames[i].crop(box=(bbox[0]-padding, 
                                                       bbox[1]-padding, 
                                                       bbox[2]+padding, 
                                                       bbox[3]+padding))
                            
                            # Se desejado, converte a região do rosto para YCrCb
                            if channel == 'luma':
                                face = cv2.cvtColor(np.array(face), cv2.COLOR_RGB2YCrCb)
                                face = face[:,:,0] # Pega apenas o canal Y
                                face = Image.fromarray(face)

                            # Se desejado, aplica um resize
                            if size > 0:
                                face = face.resize((size, size))
                            # face_count serve para não ocorrer sobrescrição de mais de um rosto por frame
                            face_count += 1
                            # Cria o path para o rosto atual
                            path = os.path.join(class_path + '/{}_batchframe{}_frameface{}_face{}.jpg'.format(video_name, _, i + 1, face_count))
                            # Salva o rosto na pasta correta. Observe que ele salva apenas face[:, :, 0] no caso do formato YCrCb (canal 'Y' = luma)
                            face.save(path)

            frames = []

    # Solta o objeto do VideoCapture
    cap.release()
    # Destrói as janelas atualmente ativas
    cv2.destroyAllWindows()

# 2° passo: Observar alguns vídeos e ajustar o threshold do MTCNN para a detecção de rostos

Definimos inicialmente a coleta de uma pasta aleatória e criamos um objeto Videos para lidar com o acesso aos vídeos.

Para acessar os vídeos, cria-se um generator que devolve, a cada acesso, um video_path, um video_name e uma label

In [7]:
# Coletamos uma pasta aleatória de folders
random_folder = random.choice(folders) + '/'
folder_path = './Kaggle Dataset/' + random_folder

# Instaciamos uma objeto da classe Videos aplicado ao path obtido
videos = Videos(folder_path)
videos_generator = videos.getAllVideosPath()

Ajustamos os thresholds e utilizamos a função de mostrar os vídeos, verificando visualmente se ele se comporta bem na maioria dos casos.

In [8]:
# Margin não faz diferença se o método .detect() for utilizado
IMAGE_SIZE = 224
MARGIN = 0
MIN_FACE_SIZE = 100
THRESHOLDS = [0.78, 0.78, 0.78]
POST_PROCESS = False
SELECT_LARGEST = False
KEEP_ALL = True
DEVICE = device

# ----------------------------------

mtcnn = MTCNN(image_size=IMAGE_SIZE,
              margin=MARGIN, 
              min_face_size=MIN_FACE_SIZE, 
              thresholds=THRESHOLDS,
              post_process=POST_PROCESS,
              select_largest=SELECT_LARGEST, 
              keep_all=KEEP_ALL, 
              device=device)

#### Vídeo Falso (Provavelmente, pois é aleatório)

In [9]:
video_path, video_name, label = next(videos_generator)

# video_path: Caminho para o vídeo
# label: string "REAL" ou "FAKE"
# size: se > 0, tornará o rosto um recorte quadrado de dimensões (size, size)
# channel: se for passado 'luma', irá mostrar apenas o canal Y (luma) do rosto.
showVideo(video_path, label=label, padding=20, size=-1, channel=None)

#### Vídeo Real

In [10]:
video_path_real, video_name_real, label_real = videos.getRealVideo(video_name)

# Podemos passar um padding para verificar a quantidade desejada de recorte ao redo do rosto detectado
showVideo(video_path_real, label=label_real, padding=20, size=-1, channel=None)

# 3° Passo: Testar o loop para 1 folder apenas

Selecionamos o primeiro folder aleatoriamente.

In [254]:
random_folder = random.choice(folders) + '/'
folder_path = './Kaggle Dataset/' + random_folder

# Instaciamos uma objeto da classe Videos aplicado ao path obtido
videos = Videos(folder_path)
videos_generator = videos.getAllVideosPath()

Realizamos um loop utilizando a função saveCropFaces, que será chamada uma vez por vídeo. Com isso, esperamos e conferimos a pasta para ver se os arquivos estão corretamente lá.

In [397]:
BATCH_SIZE = 20
PADDING = 10
SIZE = -1
CHECK_EVERY_FRAME = 15
CHANNEL = None
FOLDER = 'Dataset provisório'
SIZE_FOLDER = 'no-resize-color'

# -------------------------------------------------------------------------
init = time.time()
print("-------------- Início do folder {} --------------".format(folder))
videos_quantity = len(videos.video_files)
percentage = 5
print_every = int(videos_quantity / (100/percentage))
    
for n_video, VIDEO_DATA in enumerate(videos_generator):
    
    saveCropFaces(*VIDEO_DATA, 
                  batch_size=BATCH_SIZE, 
                  padding=PADDING, 
                  size=SIZE, 
                  check_every_frame=CHECK_EVERY_FRAME, 
                  channel=CHANNEL, 
                  folder=FOLDER,
                  size_folder=SIZE_FOLDER)
    
    if n_video % print_every == 0:
        print("{}: {:.2f}%...".format(folder, round(n_video / videos_quantity * 100)))
        
end = time.time()
total = end - init
print("Tempo que levou para a conclusão de {} vídeos: {:.0f}:{:.0f} min".format(len(os.listdir(folder_path)), int(total/60), total % 60))

-------------- Início do folder dfdc_train_part_0 --------------
dfdc_train_part_0: 0.00%...
dfdc_train_part_0: 5.00%...
dfdc_train_part_0: 10.00%...
dfdc_train_part_0: 15.00%...
dfdc_train_part_0: 20.00%...
dfdc_train_part_0: 25.00%...
dfdc_train_part_0: 30.00%...
dfdc_train_part_0: 35.00%...
dfdc_train_part_0: 40.00%...
dfdc_train_part_0: 45.00%...
dfdc_train_part_0: 49.00%...
dfdc_train_part_0: 54.00%...
dfdc_train_part_0: 59.00%...
dfdc_train_part_0: 64.00%...
dfdc_train_part_0: 69.00%...
dfdc_train_part_0: 74.00%...
dfdc_train_part_0: 79.00%...
dfdc_train_part_0: 84.00%...
dfdc_train_part_0: 89.00%...
dfdc_train_part_0: 94.00%...
Tempo que levou para a conclusão de 1335 vídeos: 21:5 min


Observamos que demora em torno de 22 minutos para terminar uma pasta. Depende bastante dq uantidade de vídeos na pasta. Podemos aumentar o batch_size para 30 para acelerar o processo.

# 4° Passo: Agora, podemos finalmente exportar todo o dataset de vídeos em imagens de rostos. (Pode levar algumas horas para concluir).

Definimos nossa MTCNN final

In [7]:
# Margin não faz diferença se o método .detect() for utilizado
IMAGE_SIZE = 224
MARGIN = 0
MIN_FACE_SIZE = 100
THRESHOLDS = [0.78, 0.78, 0.78]
POST_PROCESS = False
SELECT_LARGEST = False
KEEP_ALL = True
DEVICE = device

# ----------------------------------

mtcnn = MTCNN(image_size=IMAGE_SIZE,
              margin=MARGIN, 
              min_face_size=MIN_FACE_SIZE, 
              thresholds=THRESHOLDS,
              post_process=POST_PROCESS,
              select_largest=SELECT_LARGEST, 
              keep_all=KEEP_ALL, 
              device=device)

Aqui nós removemos os folder que já foram avaliados em momentos anteriores quando rodou-se o script do código. Para novos foldes será necessário escrever manualmente o nome na lista.

In [8]:
done_folders = ['dfdc_train_part_31', 
                'dfdc_train_part_10', 
                'dfdc_train_part_40', 
                'dfdc_train_part_22', 
                'dfdc_train_part_41', 
                'dfdc_train_part_21', 
                'dfdc_train_part_38', 
                'dfdc_train_part_36', 
                'dfdc_train_part_43']

recent_folders = [folder for folder in folders if folder not in done_folders]

In [9]:
pick_order = np.random.RandomState(seed=42).permutation(len(recent_folders))
pick_order

array([19, 16, 15, 26,  4, 12, 37, 27, 39,  6, 25,  9, 13, 31, 34,  8, 17,
       24,  0, 33,  5, 11,  1, 29, 21,  2, 30, 36,  3, 35, 23, 32, 10, 22,
       18, 20,  7, 14, 28, 38])

E iniciamos o loop que pode levar de algumas horas até alguns dias para terminar, dependendo do hardware. Valores importante que devem ser setados nessa etapa por fim:
- `CHECK_EVERY_FRAME`: Define de quantos em quantos frames será realizada a inferência da rede para obter o rosto. Caso o valor seja 1 a rede realizará a inferência de 1 em 1 frame, ou seja, tentará encontrar e recortar os rostos de todos os frames de todos os vídeos.
- `PADDING`: Controla a margem de recorte das imagens. Quanto maior este número, maior será a margem em volta do rosto recortado.
- `SIZE`: Controla as dimensões de saída do rosto recortado. Caso seja `-1`, o rosto será salvo nas dimensões originais obtidas. Como muitos frameworks de Deep Learning contém transformações do tipo **resize** extremamente otimizados, preferi manter o tamanho originalmente obtido permitindo maior flexibilidade com quem venha a implementar a leitura dessas imagens futuramente.
- `BATCH_SIZE`: Controla o tamanho do batch de frames que será processado de uma única vez pela MTCNN. Um maior número significa mais paralelado que significa mais rápido, porém pode ser que a memória dos hardwares não suporte valores muito elevados. Exemplo: Minha RTX 2070 com 8GB de memória suporta um máximo batch_size em torno de 30.
- `CHANNEL`: Controla o espaço de cores do rosto recortado. Caso seja `None`, será salva a imagem recortada do rosto com os 3 canais originais RGB. A única outra implementação disponível aqui é `'luma'`, que transforma a imagem para o espaço YCrCb e salva somente o canal Y (luma).
- `FOLDER`: Nome da pasta onde estará a subpasta que contenha as pastas "FAKE" e "REAL".
- `SIZE_FOLDER`: Nome da subpasta onde estarão as pastas "FAKE" e "REAL", que a função utilizará para distribuir as imagens corretamente.

In [22]:
PATH = './Kaggle Dataset/'
CHECK_EVERY_FRAME = 15
PADDING = 10
SIZE = -1
BATCH_SIZE = 20
CHANNEL = None
FOLDER = 'Faces Dataset'
SIZE_FOLDER = 'no-resize-color'

# -------------------------------------------------------------------------

for pick in pick_order:
    folder_path = PATH + recent_folders[pick]
    # Instaciamos uma objeto da classe Videos aplicado ao path obtido
    videos = Videos(folder_path)
    videos_generator = videos.getAllVideosPath()    
    
    print("-------------- Início do folder {} --------------".format(folders[pick]))
    videos_quantity = len(videos.video_files)
    percentage = 25
    print_every = int(videos_quantity / (100/percentage))
    
    init = time.time()
    for n_video, VIDEO_DATA in enumerate(videos_generator):
        
        saveCropFaces(*VIDEO_DATA, 
                      batch_size=BATCH_SIZE, 
                      padding=PADDING, 
                      size=SIZE, 
                      check_every_frame=CHECK_EVERY_FRAME, 
                      channel=CHANNEL, 
                      folder=FOLDER,
                      size_folder=SIZE_FOLDER)
        
        if n_video % print_every == 0:
            print("{}: {:.2f}%...".format(recent_folders[pick], round(n_video / videos_quantity * 100)))
        
        
    end = time.time()
    total = end - init
    print("Tempo para a conclusão do diretório: {:.0f}:{:.0f} min".format(int(total/60), total % 60))

-------------- Início do folder dfdc_train_part_26 --------------
dfdc_train_part_29: 0.00%...
dfdc_train_part_29: 25.00%...
dfdc_train_part_29: 50.00%...
dfdc_train_part_29: 75.00%...
dfdc_train_part_29: 100.00%...
Tempo para a conclusão do diretório: 40:25 min
-------------- Início do folder dfdc_train_part_23 --------------
dfdc_train_part_26: 0.00%...
dfdc_train_part_26: 25.00%...
dfdc_train_part_26: 50.00%...
dfdc_train_part_26: 75.00%...
dfdc_train_part_26: 100.00%...
Tempo para a conclusão do diretório: 35:15 min
-------------- Início do folder dfdc_train_part_22 --------------
dfdc_train_part_25: 0.00%...
dfdc_train_part_25: 25.00%...
dfdc_train_part_25: 50.00%...
dfdc_train_part_25: 75.00%...
dfdc_train_part_25: 100.00%...
Tempo para a conclusão do diretório: 36:0 min
-------------- Início do folder dfdc_train_part_32 --------------
dfdc_train_part_37: 0.00%...
dfdc_train_part_37: 25.00%...
dfdc_train_part_37: 50.00%...
dfdc_train_part_37: 75.00%...
dfdc_train_part_37: 100.00%

Aqui o processo foi interrompido pois a quantidade total de imagens de rostos recortadas já superava o meio milhão, suficiente para rodar algumas redes.